In [2]:
import tkinter
from tkinter import *
from PIL import Image, ImageTk
from tkinter import filedialog
import os
from random import randint
import numpy
import sys

def upshift(a,index,n):
    col = []
    for j in range(len(a)):
        col.append(a[j][index])
    shiftCol = numpy.roll(col,-n)
    for i in range(len(a)):
        for j in range(len(a[0])):
            if(j==index):
                a[i][j] = shiftCol[i]

def downshift(a,index,n):
    col = []
    for j in range(len(a)):
        col.append(a[j][index])
    shiftCol = numpy.roll(col,n)
    for i in range(len(a)):
        for j in range(len(a[0])):
            if(j==index):
                a[i][j] = shiftCol[i]

def rotate180(n):
    bits = "{0:b}".format(n)
    return int(bits[::-1], 2)

def helloCallBack(route):
    im = Image.open(route)
    pix = im.load()

    #Obtaining the RGB matrices
    r = []
    g = []
    b = []
    for i in range(im.size[0]):
        r.append([])
        g.append([])
        b.append([]) 
        for j in range(im.size[1]):
            rgbPerPixel = pix[i,j]
            r[i].append(rgbPerPixel[0])
            g[i].append(rgbPerPixel[1])
            b[i].append(rgbPerPixel[2])

    m = im.size[0]
    n = im.size[1]

    # Vectors Kr and Kc
    alpha = 8
    Kr = [randint(0,pow(2,alpha)-1) for i in range(m)]
    Kc = [randint(0,pow(2,alpha)-1) for i in range(n)]
    ITER_MAX = 1

    print('Vector Kr : ', Kr)
    print('Vector Kc : ', Kc)

    f = open('keysKr.txt','w+')
    #f.write('Vector Kr : \n')
    for a in Kr:
        f.write(str(a) + '\n')
    #f.write('Vector Kc : \n')
    f.close()

    f = open('keysKc.txt','w+')
    for a in Kc:
        f.write(str(a) + '\n')
    f.close()

    f = open('iterMax.txt','w+')
    #f.write('ITER_MAX : \n')
    f.write(str(ITER_MAX) + '\n')
    f.close()


    for iterations in range(ITER_MAX):
        # For each row
        for i in range(m):
            rTotalSum = sum(r[i])
            gTotalSum = sum(g[i])
            bTotalSum = sum(b[i])
            rModulus = rTotalSum % 2
            gModulus = gTotalSum % 2
            bModulus = bTotalSum % 2
            if(rModulus==0):
                r[i] = numpy.roll(r[i],Kr[i])
            else:
                r[i] = numpy.roll(r[i],-Kr[i])
            if(gModulus==0):
                g[i] = numpy.roll(g[i],Kr[i])
            else:
                g[i] = numpy.roll(g[i],-Kr[i])
            if(bModulus==0):
                b[i] = numpy.roll(b[i],Kr[i])
            else:
                b[i] = numpy.roll(b[i],-Kr[i])
        # For each column
        for i in range(n):
            rTotalSum = 0
            gTotalSum = 0
            bTotalSum = 0
            for j in range(m):
                rTotalSum += r[j][i]
                gTotalSum += g[j][i]
                bTotalSum += b[j][i]
            rModulus = rTotalSum % 2
            gModulus = gTotalSum % 2
            bModulus = bTotalSum % 2
            if(rModulus==0):
                upshift(r,i,Kc[i])
            else:
                downshift(r,i,Kc[i])
            if(gModulus==0):
                upshift(g,i,Kc[i])
            else:
                downshift(g,i,Kc[i])
            if(bModulus==0):
                upshift(b,i,Kc[i])
            else:
                downshift(b,i,Kc[i])
        # For each row
        for i in range(m):
            for j in range(n):
                if(i%2==1):
                    r[i][j] = r[i][j] ^ Kc[j]
                    g[i][j] = g[i][j] ^ Kc[j]
                    b[i][j] = b[i][j] ^ Kc[j]
                else:
                    r[i][j] = r[i][j] ^ rotate180(Kc[j])
                    g[i][j] = g[i][j] ^ rotate180(Kc[j])
                    b[i][j] = b[i][j] ^ rotate180(Kc[j])
        # For each column
        for j in range(n):
            for i in range(m):
                if(j%2==0):
                    r[i][j] = r[i][j] ^ Kr[i]
                    g[i][j] = g[i][j] ^ Kr[i]
                    b[i][j] = b[i][j] ^ Kr[i]
                else:
                    r[i][j] = r[i][j] ^ rotate180(Kr[i])
                    g[i][j] = g[i][j] ^ rotate180(Kr[i])
                    b[i][j] = b[i][j] ^ rotate180(Kr[i])


    for i in range(m):
        for j in range(n):
            pix[i,j] = (r[i][j],g[i][j],b[i][j])

    im.save("imgencrip.png")
    global label1
    image1 = ImageTk.PhotoImage(im)
    label1.configure(image = image1)
    label1.image = image1
    
    global answer 
    answer = 'imgencrip.png'
    
def Desencriptar(route):

    im = Image.open(route)
    pix = im.load()


    #Obtaining the RGB matrices
    r = []
    g = []
    b = []
    for i in range(im.size[0]):
        r.append([])
        g.append([])
        b.append([]) 
        for j in range(im.size[1]):
            rgbPerPixel = pix[i,j]
            r[i].append(rgbPerPixel[0])
            g[i].append(rgbPerPixel[1])
            b[i].append(rgbPerPixel[2])

    m = im.size[0]
    n = im.size[1]

    Kr = []
    Kc = []

    with open ('keysKr.txt', 'rt') as myfile:  # Open lorem.txt for reading text
        contents = myfile.read()              # Read the entire file to a string

    testing = contents.split()

    Kr = list(map(int, testing))

    with open ('keysKc.txt', 'rt') as myfile:  # Open lorem.txt for reading text
        contents = myfile.read()              # Read the entire file to a string

    testing = contents.split()

    Kc = list(map(int, testing))

    #print('Enter value of Kr')

    #for i in range(m):
    #	Kr.append(int(input()))

    #print('Enter value of Kc')
    #for i in range(n):
    #	Kc.append(int(input()))

    #print('Enter value of ITER_MAX')
    #ITER_MAX = int(input())


    for iterations in range(ITER_MAX):
        # For each column
        for j in range(n):
            for i in range(m):
                if(j%2==0):
                    r[i][j] = r[i][j] ^ Kr[i]
                    g[i][j] = g[i][j] ^ Kr[i]
                    b[i][j] = b[i][j] ^ Kr[i]
                else:
                    r[i][j] = r[i][j] ^ rotate180(Kr[i])
                    g[i][j] = g[i][j] ^ rotate180(Kr[i])
                    b[i][j] = b[i][j] ^ rotate180(Kr[i])
        # For each row
        for i in range(m):
            for j in range(n):
                if(i%2==1):
                    r[i][j] = r[i][j] ^ Kc[j]
                    g[i][j] = g[i][j] ^ Kc[j]
                    b[i][j] = b[i][j] ^ Kc[j]
                else:
                    r[i][j] = r[i][j] ^ rotate180(Kc[j])
                    g[i][j] = g[i][j] ^ rotate180(Kc[j])
                    b[i][j] = b[i][j] ^ rotate180(Kc[j])
        # For each column
        for i in range(n):
            rTotalSum = 0
            gTotalSum = 0
            bTotalSum = 0
            for j in range(m):
                rTotalSum += r[j][i]
                gTotalSum += g[j][i]
                bTotalSum += b[j][i]
            rModulus = rTotalSum % 2
            gModulus = gTotalSum % 2
            bModulus = bTotalSum % 2
            if(rModulus==0):
                downshift(r,i,Kc[i])
            else:
                upshift(r,i,Kc[i])
            if(gModulus==0):
                downshift(g,i,Kc[i])
            else:
                upshift(g,i,Kc[i])
            if(bModulus==0):
                downshift(b,i,Kc[i])
            else:
                upshift(b,i,Kc[i])

        # For each row
        for i in range(m):
            rTotalSum = sum(r[i])
            gTotalSum = sum(g[i])
            bTotalSum = sum(b[i])
            rModulus = rTotalSum % 2
            gModulus = gTotalSum % 2
            bModulus = bTotalSum % 2
            if(rModulus==0):
                r[i] = numpy.roll(r[i],-Kr[i])
            else:
                r[i] = numpy.roll(r[i],Kr[i])
            if(gModulus==0):
                g[i] = numpy.roll(g[i],-Kr[i])
            else:
                g[i] = numpy.roll(g[i],Kr[i])
            if(bModulus==0):
                b[i] = numpy.roll(b[i],-Kr[i])
            else:
                b[i] = numpy.roll(b[i],Kr[i])

    for i in range(m):
        for j in range(n):
            pix[i,j] = (r[i][j],g[i][j],b[i][j])

    im.save('imgdecrip.png')
    global label1
    image1 = ImageTk.PhotoImage(im)
    label1.configure(image = image1)
    label1.image = image1
    global answer 
    answer = 'imgdecrip.png'
    
################################ Main ################################

ITER_MAX = 1

root = Tk()

root.title('Encriptador/Desencriptador')

# Build a list of tuples for each file type the file dialog should display
my_filetypes = [('png files', '.png')]

# Ask the user to select a single file name.
answer = filedialog.askopenfilename(parent=root,
                                    initialdir=os.getcwd(),
                                    title="Please select a file:",
                                    filetypes=my_filetypes)
print(answer)

# Create a photoimage object of the image in the path
image1 = Image.open(answer)
test = ImageTk.PhotoImage(image1)

label1 = tkinter.Label(image=test)
label1.image = test

# Position image
label1.place(x=0, y=0)

width = str(image1.width)
height = str(image1.height + 50)
root.geometry(width + "x" + height)

button = Button(root, text ="Encriptar", command = lambda: helloCallBack(answer))
button.pack()
button.place(relx=0.25, rely=0.95, anchor=CENTER)

button2 = Button(root, text ="Desencriptar", command = lambda: Desencriptar(answer))
button2.pack()
button2.place(relx=0.75, rely=0.95, anchor=CENTER)

root.mainloop()

C:/Users/kevs_/Documents/6to Semestre/Redes/Encrip2/cuba.png
